In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

latent_dim = 512  
num_samples = 1000  
data_path = 'example.txt'  

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    splitted = line.split('#')
    print(splitted)
    input_text = splitted[0]
    target_text = splitted[1]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=10, validation_split=0.2)


2024-02-06 11:58:13.538381: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 11:58:13.564388: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 11:58:13.564412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 11:58:13.565108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 11:58:13.569142: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 11:58:13.569606: I tensorflow/core/platform/cpu_feature_guard.cc:1

["How's the weather today?", "It's sunny and warm outside."]
['Tell me about your family.', 'I have two siblings and loving parents.']
["What's your favorite type of music?", 'I enjoy listening to classical music.']
['Did you watch the latest movie?', 'Yes, I watched it and really enjoyed it.']
["What's your go-to comfort food?", 'I love indulging in macaroni and cheese.']
['Have you ever been to a concert?', "Yes, I've been to several concerts, and they're always exciting."]
['Do you have any pets?', 'Yes, I have a playful Labrador named Max.']
["What's your earliest childhood memory?", 'I remember playing in the backyard with my friends.']
['Are you a morning person or a night owl?', 'I am more productive in the morning.']
['Tell me about a hobby you enjoy.', 'I love painting and expressing myself through art.']
['How do you handle stress?', 'I usually take a break, go for a walk, or practice deep breathing.']
["What's your favorite season?", 'I adore the beauty of autumn with its co

Epoch 1/10
13/13 [==============================] - 8s 493ms/step - loss: 2.0136 - accuracy: 0.0664 - val_loss: 1.8351 - val_accuracy: 0.1082
Epoch 2/10
13/13 [==============================] - 6s 433ms/step - loss: 1.7942 - accuracy: 0.0933 - val_loss: 1.7993 - val_accuracy: 0.0950
Epoch 3/10
13/13 [==============================] - 6s 429ms/step - loss: 1.7675 - accuracy: 0.0948 - val_loss: 1.7741 - val_accuracy: 0.0971
Epoch 4/10
13/13 [==============================] - 6s 449ms/step - loss: 1.7492 - accuracy: 0.0955 - val_loss: 1.7679 - val_accuracy: 0.0973
Epoch 5/10
13/13 [==============================] - 6s 442ms/step - loss: 1.7510 - accuracy: 0.0951 - val_loss: 1.7883 - val_accuracy: 0.0905
Epoch 6/10
13/13 [==============================] - 6s 438ms/step - loss: 1.7477 - accuracy: 0.0945 - val_loss: 1.7529 - val_accuracy: 0.0984
Epoch 7/10
13/13 [==============================] - 6s 432ms/step - loss: 1.7288 - accuracy: 0.0967 - val_loss: 1.7454 - val_accuracy: 0.0961
Epoch 

In [2]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [3]:
def generate_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    target_seq[0, 0, target_token_index['\t']] = 1.0

    stop_condition = False
    generated_sequence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        generated_sequence += sampled_char

        if sampled_char == '\n' or len(generated_sequence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]

    return generated_sequence


In [4]:
input_sequence = encoder_input_data[0:1]

generated_sequence = generate_sequence(input_sequence)

print("Input Sequence:", input_texts[0])
print("Generated Sequence:", generated_sequence)


1/1 [==============================] - 0s 12ms/step
Input Sequence: How's the weather today?
Generated Sequence: tttsuunn                                                                                        
